In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [5]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="test_ci")

for v in ['divo', 'bouaké', 'anyama', 'san-pédro', 'daloa', 'gagnoa',
       'abidjan', 'man', 'grand-bassam', 'yamoussoukro', 'korhogo']:
    loc = geolocator.geocode(v, country_codes="ci")
    print(v, "->", loc)
    if loc:
        # affiche tout le raw pour voir la structure réelle
        print("RAW:", loc.raw)            
        addr = loc.raw.get("address", {}) 
        print("ADDRESS:", addr)
        print("\n")

divo -> Divo, Lôh-Djiboua, Gôh-Djiboua, Côte d’Ivoire
RAW: {'place_id': 277785976, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 443389991, 'lat': '5.8405332', 'lon': '-5.3549049', 'class': 'place', 'type': 'city', 'place_rank': 16, 'importance': 0.37320859947062124, 'addresstype': 'city', 'name': 'Divo', 'display_name': 'Divo, Lôh-Djiboua, Gôh-Djiboua, Côte d’Ivoire', 'boundingbox': ['5.6805332', '6.0005332', '-5.5149049', '-5.1949049']}
ADDRESS: {}


bouaké -> Bouaké, Gbêkê, Vallée du Bandama, Côte d’Ivoire
RAW: {'place_id': 277030935, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 27565008, 'lat': '7.6890212', 'lon': '-5.0283552', 'class': 'place', 'type': 'city', 'place_rank': 16, 'importance': 0.5063191432037928, 'addresstype': 'city', 'name': 'Bouaké', 'display_name': 'Bouaké, Gbêkê, Vallée du Bandama, Côte d’Ivoire', 'boundingbox': ['7.5290212', 

In [4]:
df = pd.DataFrame({"ville": ['divo', 'bouaké', 'anyama', 'san-pédro', 'daloa', 'gagnoa',
       'abidjan', 'man', 'grand-bassam', 'yamoussoukro', 'korhogo']})

# Initialiser le géocodeur Nominatim (OpenStreetMap)
geolocator = Nominatim(user_agent="mon_app_geocodage_ci")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def get_region_from_city_ci(city_name):
    if pd.isna(city_name):
        return None

    loc = geocode(city_name, country_codes="ci")
    if not loc:
        return None

    # Récupérer le display_name depuis le dict brut
    disp = loc.raw.get("display_name")
    if not disp:
        return None

    # Exemple: "Daloa, Haut-Sassandra, Sassandra-Marahoué, Côte d’Ivoire"
    parts = [p.strip() for p in disp.split(",")]

    # Enlever le pays (dernier élément)
    if parts and parts[-1].lower().startswith(("côte d’ivoire", "cote d ivoire")):
        parts = parts[:-1]

    # Heuristique simple : on prend l'avant-dernier élément comme région
    # Daloa -> [..., 'Haut-Sassandra', 'Sassandra-Marahoué'] -> 'Sassandra-Marahoué'
    # Grand-Bassam -> [..., 'Sud-Comoé', 'Comoé'] -> 'Comoé'
    if len(parts) >= 3:
        region = parts[-2]
        return region

    return None
# Appliquer sur la colonne ville
df["region"] = df["ville"].apply(get_region_from_city_ci)

print(df)

           ville          region
0           divo     Lôh-Djiboua
1         bouaké           Gbêkê
2         anyama            None
3      san-pédro            None
4          daloa  Haut-Sassandra
5         gagnoa             Gôh
6        abidjan            None
7            man          Tonkpi
8   grand-bassam       Sud-Comoé
9   yamoussoukro            None
10       korhogo            Poro
